In [0]:
import cv2
import argparse
import numpy as np
!pip install opencv-wrapper
import opencv_wrapper as cvw
from skimage.filters import threshold_local
import json


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

!pip install -U git+https://github.com/madmaze/pytesseract.git

import pytesseract
!sudo apt install tesseract-ocr
!pip install colorama

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
  Cloning https://github.com/madmaze/pytesseract.git to /tmp/pip-req-build-r0fojitl
  Running command git clone -q https://github.com/madmaze/pytesseract.git /tmp/pip-req-build-r0fojitl
  Created wheel for pytesseract: filename=pytesseract-0.3.4-py2.py3-none-any.whl size=13433 sha256=9c596266f8a900a1907cc98cdcd63536c3cc280d8ff26efe562d7fccb3f48e8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-1f4ycnag/wheels/be/2a/a1/a40dbc7e579dffb2be8bbc3243c491de2d132899309f008b1f
Successfully built pytesseract
  Found existing installation: pytesseract 0.3.4
    Uninstalling pytesseract-0.3.4:
      Successfully uninstalled pytesseract-0.3.4
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgrade

In [0]:

# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required = True, help = "Path to the image to be scanned")
# ap.add_argument("-o", "--output", required = True, help = "Path to the image to be scanned")
# args = vars(ap.parse_args())


def get_acc(directory, path):
    font     = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.5
    fontColor  = (255,0,0)
    lineType = 1
    path = directory+path
    # path = args['image']
    # op_path = args['output']

    op_path = directory
    if op_path[-1]!='/':
    	op_path.append('/')


    #Threshold
    image = cv2.imread(path)

    height,width,channel = image.shape

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    T = threshold_local(gray, 15, offset = 6, method = "gaussian") # generic, mean, median, gaussian
    thresh = (gray > T).astype("uint8") * 255
    thresh = ~thresh

    #cv2.imwrite(op_path+'threshold.png', thresh)

    #Dilation
    kernel =np.ones((1,1), np.uint8)
    ero = cv2.erode(thresh, kernel, iterations= 1)
    img_dilation = cv2.dilate(ero, kernel, iterations=1)
    #cv2.imwrite(op_path+'dilated.png', img_dilation)

    # Remove noise
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(img_dilation, None, None, None, 8, cv2.CV_32S)
    sizes = stats[1:, -1] #get CC_STAT_AREA component
    final = np.zeros((labels.shape), np.uint8)
    for i in range(0, nlabels - 1):
        if sizes[i] >= 10:   #filter small dotted regions
            final[labels == i + 1] = 255
    #cv2.imwrite(op_path+'final.png', final)
    #Find contours
    kern = np.ones((5,15), np.uint8)
    img_dilation = cv2.dilate(final, kern, iterations = 1)
    #cv2.imwrite(op_path+'contours.png', img_dilation)
    contours, hierarchy = cv2.findContours(img_dilation, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    # Map contours to bounding rectangles, using bounding_rect property
    rects = map(lambda c: cv2.boundingRect(c), contours)
    # Sort rects by top-left x (rect.x == rect.tl.x)
    sorted_rects = sorted(rects, key =lambda r: r[0])
    sorted_rects = sorted(sorted_rects, key =lambda r: r[1])

    tt = image.copy()
    dictionary = {}
    etfo=''
    for i,rect in enumerate(sorted_rects):
        temp_dic = {}
        x,y,w,h = rect
        if(w<20 or h<20):
            continue
        temp_dic['coords'] = [x,y,w,h]
        words = []
        temp = tt[y:y+h, x:x+w]
        #cv2.imwrite('/content/gdrive/My Drive/ResearchPaper/AIESI_complete_pipeline/temp/'+str(i)+'.png',temp)
        temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
        hi = pytesseract.image_to_data(temp, config=r'--psm 6')
        hi = hi.split()
        ind = 22
        while(True):
            if (ind>len(hi)):
                break
            if(int(hi[ind])==-1):
                ind+=11
            else:
                #cv2.putText(image,hi[ind]+','+hi[ind+1], (x,y), font, fontScale,fontColor,lineType)
                tem = {}
                tem['confidence'] = hi[ind]
                tem['text'] = hi[ind+1]
                etfo=etfo+hi[ind+1]
                etfo=etfo+" "
                x+=len(hi[ind+1])*20
                ind+=12
                words.append(tem)
        temp_dic['words'] = words
        etfo=etfo+'\n'
        #cvw.rectangle(image, rect, cvw.Color.GREEN, thickness=1)
        dictionary[i] = temp_dic


    cv2.imwrite(op_path+"result.png", image)
    return json.dumps(dictionary),etfo


In [0]:

gson_data, etfo = get_acc('/content/gdrive/My Drive/ResearchPaper/AIESI_complete_pipeline/sample/','hello.png')
print(gson_data)


{"0": {"coords": [278, 27, 145, 76], "words": [{"confidence": "93", "text": "chay"}]}, "1": {"coords": [165, 34, 104, 44], "words": [{"confidence": "95", "text": "tan"}]}, "2": {"coords": [411, 43, 127, 66], "words": [{"confidence": "65", "text": "yee"}]}, "9": {"coords": [303, 155, 169, 32], "words": [{"confidence": "93", "text": "COPY"}, {"confidence": "93", "text": "***"}]}, "10": {"coords": [232, 156, 70, 22], "words": [{"confidence": "0", "text": "KK"}]}, "12": {"coords": [141, 192, 346, 32], "words": [{"confidence": "85", "text": "OJC"}, {"confidence": "96", "text": "MARKETING"}, {"confidence": "96", "text": "SDN"}]}, "13": {"coords": [489, 195, 76, 29], "words": [{"confidence": "96", "text": "BHD"}]}, "17": {"coords": [213, 236, 281, 28], "words": [{"confidence": "94", "text": "ROC"}, {"confidence": "90", "text": "NO:"}, {"confidence": "84", "text": "538358-H"}]}, "19": {"coords": [179, 274, 347, 33], "words": [{"confidence": "93", "text": "NO"}, {"confidence": "80", "text": "2"

In [0]:
import argparse
import torch
import json
from torch import nn

In [0]:
class MyModel0(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=2, bidirectional=True)
        self.linear = nn.Linear(hidden_size * 2, 5)

    def forward(self, inpt):
        embedded = self.embed(inpt)
        feature, _ = self.lstm(embedded)
        oupt = self.linear(feature)
        return oupt


### Hyper- parameters

In [0]:
device= 'cpu'
hidden_size = 256

device= torch.device('cpu')

In [0]:
!pip install colorama

In [0]:
import json
import os
import random
from os import path
from string import ascii_uppercase, digits, punctuation

import colorama
import numpy
import regex


In [0]:
VOCAB= ascii_uppercase+digits+punctuation+" \t\n"


In [0]:
print(etfo)

chay 
tan 
yee 
COPY *** 
KK 
OJC MARKETING SDN 
BHD 
ROC NO: 538358-H 
NO 2 & 4, JALAN BAYU 4, 
BANDAR SERI ALAM, 
81750 MASAI, JOHOR 
Tel:07-388 2218 Fax:07-388 8218 
Email: ng@ojcgroup.cam 
TAX INVOICE 
Invoice No 
* PEGIV-1030765 
: 15/01/2019 
11:05:16 AM 
Date 
: NG CHUAN MIN 
Cashier 
Sales Persor : FATIN 
Bill To 
: THE PEAK QUARRY WORKS 
Address 
_Description ss Oty, Price = Amount 
000000111 
193,00 
1 
193.00 SR 
KINGS SAFETY SHOES KWD 805 
“Oty: 1. TotalExclude GST: + +193.00 
Qty: 1 
133.00 
Total GST @6%: 
0.00 
Total Inclusive GST: 
193.00 
ee eae OU EL AM ber acces OO t. 
TOTAL: 
193,00 
VISA CARD 
(9-0) 
fons 3-00 
WO0C0O0K0000K4 3 1B 
Approval Code:000 
Goods Sold Are Not Returnable & Refundable 
*+*+4+Thank You. Please Come Again,+*** 



In [0]:
temp_text=''
for i in range(664):
    temp_text=temp_text+etfo[i]
etfo=temp_text

In [0]:
# conver lower text to uppper text in ETFO
etfo= etfo.upper()
print(etfo)



CHAY 
TAN 
YEE 
COPY *** 
KK 
OJC MARKETING SDN 
BHD 
ROC NO: 538358-H 
NO 2 & 4, JALAN BAYU 4, 
BANDAR SERI ALAM, 
81750 MASAI, JOHOR 
TEL:07-388 2218 FAX:07-388 8218 
EMAIL: NG@OJCGROUP.CAM 
TAX INVOICE 
INVOICE NO 
* PEGIV-1030765 
: 15/01/2019 
11:05:16 AM 
DATE 
: NG CHUAN MIN 
CASHIER 
SALES PERSOR : FATIN 
BILL TO 
: THE PEAK QUARRY WORKS 
ADDRESS 
_DESCRIPTION SS OTY, PRICE = AMOUNT 
000000111 
193,00 
1 
193.00 SR 
KINGS SAFETY SHOES KWD 805 
“OTY: 1. TOTALEXCLUDE GST: + +193.00 
QTY: 1 
133.00 
TOTAL GST @6%: 
0.00 
TOTAL INCLUSIVE GST: 
193.00 
EE EAE OU EL AM BER ACCES OO T. 
TOTAL: 
193,00 
VISA CARD 
(9-0) 
FONS 3-00 
WO0C0O0K0000K4 3 1B 
APP


In [0]:
def get_test_data():
    
    text = etfo
    text_tensor = torch.zeros(len(text), 1, dtype=torch.long)
    text_tensor[:, 0] = torch.LongTensor([VOCAB.find(c) for c in text])

    return text_tensor.to(device)

text_tensor = get_test_data()


In [0]:
print(text_tensor.shape)

torch.Size([664, 1])


In [0]:
def pred_to_dict(text, pred, prob):
    res = {"company": ("", 0), "date": ("", 0), "address": ("", 0), "total": ("", 0)}
    keys = list(res.keys())

    seps = [0] + (numpy.nonzero(numpy.diff(pred))[0] + 1).tolist() + [len(pred)]
    for i in range(len(seps) - 1):
        pred_class = pred[seps[i]] - 1
        if pred_class == -1:
            continue

        new_key = keys[pred_class]
        new_prob = prob[seps[i] : seps[i + 1]].max()
        if new_prob > res[new_key][1]:
            res[new_key] = (text[seps[i] : seps[i + 1]], new_prob)

    return {k: regex.sub(r"[\t\n]", " ", v[0].strip()) for k, v in res.items()}


In [0]:
print(text_tensor.shape)
for i in range(len(text_tensor)-1):
  if text_tensor[i]<0 or text_tensor[i]>70:
    text_tensor = torch.cat([text_tensor[0:i], text_tensor[i+1:]])

torch.Size([664, 1])


In [0]:

def test():
    
    model = MyModel0(len(VOCAB), 16, hidden_size).to(device)
    # dataset = MyDataset(None, args.device, test_path="/content/gdrive/My Drive/ResearchPaper/KIPE/data/test_dict.pth")

    model.load_state_dict(torch.load("/content/gdrive/My Drive/ResearchPaper/KIPE/src/model.pth"))

    model.eval()
  
    with torch.no_grad():
            oupt = model(text_tensor)
            prob = torch.nn.functional.softmax(oupt, dim=2)
            prob, pred = torch.max(prob, dim=2)
            prob = prob.squeeze().cpu().numpy()
            pred = pred.squeeze().cpu().numpy()
            real_text = etfo
            result = pred_to_dict(real_text, pred, prob)

            with open("/content/gdrive/My Drive/ResearchPaper/AIESI_complete_pipeline/result" + 'result' + ".json", "w", encoding="utf-8") as json_opened:
                json.dump(result, json_opened, indent=4)

            print(result)
            #print(key)


if __name__ == "__main__":
    test()

{'company': 'KK  OJC MARKETING SDN  BHD', 'date': '01', 'address': 'NO 2 & 4, JALAN BAYU 4,  BANDAR SERI ALAM,  81750 MASAI, JOHOR', 'total': '133.0'}
